In [2]:
import torch

In [3]:
import numpy as np

In [4]:
from fastai.text.all import *
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

In [5]:
pretrained_weights = 'gpt2'
tokenizer = GPT2TokenizerFast.from_pretrained(pretrained_weights)
model = GPT2LMHeadModel.from_pretrained(pretrained_weights)

In [6]:
path="G:\ML_FCC\LLM from Scratch\input_data"

In [7]:
poems = get_text_files(path, folders = ['forms','topics'])
print("There are",len(poems),"poems in the dataset")

There are 20657 poems in the dataset


In [8]:
ballads = get_text_files(path+'/forms', folders = ['ballad'])
print("There are",len(ballads),"ballads in the dataset")

There are 100 ballads in the dataset


In [9]:
txt = poems[0].open().read(); 
print(txt)

2 ABC of H.k. and China revised vision.
Barrels tears are wines and salts.
With a whisk on goody tails!
Wiggle maces to fix the heads.
Heads in jack on boxes are ceased.
Cry to paranoid truly bosses.
Bosses are jokers take your boys.
Studs are bogs with fire apples.
True predicates worth cases.â€™
Descents wash in badly bands.
Wholly sales are smart with cats.
Who got tenth honors in China?
Homage grand to play and plays!
Trim the times of hearts then cry.
Tanks in steels but voice wail.
Bossy dragged by tails that whisked.
Go very timid and love the wise.
Hands are lent but laws are ends.
Cases on courts are borrowed lands.
Length long with treads to retch!
Straps on times and watch here.
Arrays tanks but all are men.
Cross all suctions steal the ends.
Cave on minds are cages on objects.
Rouser rockets powers holes.
Confine curses to stop our wounds.
Whirl your bodies and jump on grounds.
Crouch of soldiers after kicks with flings.
Block one leg and hit the middle.
Cauchy3 know the tr

In [10]:
ballad_texts = []

# Iterate over each file and read its content
for file_path in ballads:
    with open(file_path, 'rb') as file:
        try:
            content = file.read().decode('utf-8')  # Use the appropriate encoding
            ballad_texts.append(content)
        except UnicodeDecodeError:
            print(f"Error decoding file: {file_path}")

# Convert the list of ballad texts into a numpy array
ballads_array = np.array(ballad_texts)

In [11]:
def flatten(A):
    rt = []
    for i in A:
        if isinstance(i,list): rt.extend(flatten(i))
        else: rt.append(i)
    return rt

In [12]:
all_ballads = flatten(ballads_array)

In [13]:
class TransformersTokenizer(Transform):
    def __init__(self, tokenizer): self.tokenizer = tokenizer
    def encodes(self, x): 
        toks = self.tokenizer.tokenize(x)
        return tensor(self.tokenizer.convert_tokens_to_ids(toks))
    def decodes(self, x): return TitledStr(self.tokenizer.decode(x.cpu().numpy()))

In [14]:
splits = [range_of(70), range(100)]
tls = TfmdLists(all_ballads, TransformersTokenizer(tokenizer), splits=splits, dl_type=LMDataLoader)

In [15]:
show_at(tls.train, 0)

The burden of hard hitting. Slug away
Like Honus Wagner or like Tyrus Cobb.
Else fandom shouteth: "Who said you could play?
Back to the jasper league, you minor slob!"
Swat, hit, connect, line out, goet on the job.
Else you shall feel the brunt of fandom's ire
Biff, bang it, clout it, hit it on the knob -
This is the end of every fan's desire.
The burden of good pitching. Curved or straight.
Or in or out, or haply up or down,
To puzzle him that standeth by the plate,
To lessen, so to speak, his bat-renown:
Like Christy Mathewson or Miner Brown,
So pitch that every man can but admire
And offer you the freedom of the town -
This is the end of every fan's desire.
The burden of loud cheering. O the sounds!
The tumult and the shouting from the throats
Of forty thousand at the Polo Grounds
Sitting, ay, standing sans their hats and coats.
A mighty cheer that possibly denotes
That Cub or Pirate fat is in the fire;
Or, as H. James would say, We've got their goats -
This is the end of every fan'

In [16]:
bs,sl = 4,256
dls = tls.dataloaders(bs=bs, seq_len=sl)

Token indices sequence length is longer than the specified maximum sequence length for this model (1327 > 1024). Running this sequence through the model will result in indexing errors


In [17]:
dls.show_batch(max_n=2)

,text,text_
0,I have walked through tougher Harlem where few strangers dare to go\r\nAnd I've been in London City in the rain and in the snow\r\nAnd I've worked in inner Melbourne in the searing summer heat\r\nAnd believe me if I tell you I have earned the bread I eat.\r\nI have laboured in deep trenches with my life I've took a dare\r\nAnd I've worked in cherry pickers ninety foot up in the air\r\nAnd the hands of time keep turning and the years go quickly by\r\nAnd the man who lives on welfare is still better off than I.\r\nAnd who needs the tag of good worker it's no big deal anyway\r\nHe's a wiser and better off man who sits at home all day\r\nAnd his conscience doesn't prick him isn't he the lucky one\r\nAnd must I be one great idiot to go labouring in the sun.\r\nI was low in social ladder and I still am way,have walked through tougher Harlem where few strangers dare to go\r\nAnd I've been in London City in the rain and in the snow\r\nAnd I've worked in inner Melbourne in the searing summer heat\r\nAnd believe me if I tell you I have earned the bread I eat.\r\nI have laboured in deep trenches with my life I've took a dare\r\nAnd I've worked in cherry pickers ninety foot up in the air\r\nAnd the hands of time keep turning and the years go quickly by\r\nAnd the man who lives on welfare is still better off than I.\r\nAnd who needs the tag of good worker it's no big deal anyway\r\nHe's a wiser and better off man who sits at home all day\r\nAnd his conscience doesn't prick him isn't he the lucky one\r\nAnd must I be one great idiot to go labouring in the sun.\r\nI was low in social ladder and I still am way
1,"body\r\nYou should not do this thing.\r\n""Take off, take off, those shoes of pride,\r\nCarry them whence they came;\r\nYour Captains saw your insolence,\r\nAnd they shall see your shame.""\r\nWhen Mehtab Singh came to the door\r\nHis shoes they burned his hand,\r\nFor there in long and silent lines\r\nHe saw the Captains stand.\r\nWhen Mehtab Singh rode from the gate\r\nHis chin was on his breast:\r\nThe captains said, ""When the strong command\r\nObedience is best.""'T was Fultah Fisher's boarding-house,\r\nWhere sailor-men reside,\r\nAnd there were men of all the ports\r\nFrom Mississip to Clyde,\r\nAnd regally they spat and smoked,\r\nAnd fearsomely they lied.\r\nThey lied about the purple Sea\r\nThat gave them scanty bread,\r\nThey lied about the Earth beneath,\r\nThe Heavens overhead,\r\nFor they had looked too often on\r\nBlack rum when that was red.\r\nThey told their tales of wreck and wrong,\r\nOf shame and lust and fraud,","\r\nYou should not do this thing.\r\n""Take off, take off, those shoes of pride,\r\nCarry them whence they came;\r\nYour Captains saw your insolence,\r\nAnd they shall see your shame.""\r\nWhen Mehtab Singh came to the door\r\nHis shoes they burned his hand,\r\nFor there in long and silent lines\r\nHe saw the Captains stand.\r\nWhen Mehtab Singh rode from the gate\r\nHis chin was on his breast:\r\nThe captains said, ""When the strong command\r\nObedience is best.""'T was Fultah Fisher's boarding-house,\r\nWhere sailor-men reside,\r\nAnd there were men of all the ports\r\nFrom Mississip to Clyde,\r\nAnd regally they spat and smoked,\r\nAnd fearsomely they lied.\r\nThey lied about the purple Sea\r\nThat gave them scanty bread,\r\nThey lied about the Earth beneath,\r\nThe Heavens overhead,\r\nFor they had looked too often on\r\nBlack rum when that was red.\r\nThey told their tales of wreck and wrong,\r\nOf shame and lust and fraud,\r"


In [ ]:
#finetuning_1

In [18]:
class DropOutput(Callback):
    def after_pred(self): self.learn.pred = self.pred[0]

In [19]:
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), cbs=[DropOutput], metrics=Perplexity()).to_fp16()

In [20]:
learn.validate()

F:\Python\Lib\site-packages\torch\amp\autocast_mode.py:204: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
F:\Python\Lib\site-packages\torch\cuda\amp\grad_scaler.py:120: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


(#2) [5.849992752075195,347.23187255859375]

In [1]:
learn.lr_find()

NameError: name 'learn' is not defined

In [ ]:
prompt = 'love is ridiculous'
prompt_ids = tokenizer.encode(prompt)
inp = torch.tensor(prompt_ids)[None]  # No need for .cuda() here

print(inp.shape)

In [ ]:
preds = learn.model.generate(inp, max_length=60, num_beams=5, no_repeat_ngram_size=2, early_stopping=True)
print("Output:\n" + 100 * '-')
print(tokenizer.decode(preds[0].cpu().numpy(), skip_special_tokens=True))

In [ ]:

import gradio as gr

def generate_text(prompt):
    prompt_ids = tokenizer.encode(prompt)
    inp = torch.tensor(prompt_ids)[None]

    preds = model.generate(
        inp,
        max_length=100,
        num_beams=5,
        no_repeat_ngram_size=2,
        early_stopping=True
    )

    generated_text = tokenizer.decode(preds[0], skip_special_tokens=True)
    return generated_text

iface = gr.Interface(
    fn=generate_text,
    inputs=gr.inputs.Textbox(prompt="Enter a prompt"),
    outputs="text",
    live=True,
    title="GPT-2 Text Generation",
    description="Generate text using GPT-2 model"
)

if __name__ == "__main__":
    iface.launch()


In [ ]:
#finetuning_2

In [ ]:
class CustomCallback(Callback):
    def after_epoch(self):
        print(f"Epoch {self.epoch}: Train Loss - {self.learn.recorder.losses[-1]:.4f}, Perplexity - {self.learn.recorder.metrics[-1]:.2f}")
lr_scheduler = ReduceLROnPlateau(monitor='valid_loss', factor=0.1, patience=2, min_lr=1e-6)
learn = language_model_learner(
    dls, 
    AWD_LSTM, 
    loss_func=CrossEntropyLossFlat(), 
    cbs=[DropOutput, CustomCallback],
    metrics=Perplexity(),
    opt_func=AdamW,
    lr=1e-3
).to_fp16()
learn.fit_one_cycle(5, lr_max=slice(1e-3))
learn.lr_find()
learn.fit_one_cycle(5, lr_max=slice(1e-5, 1e-4), cbs=lr_scheduler)
learn.save("fine_tuned_model")
prompt = "Once upon a time"
generated_text = learn.predict(prompt, n_words=50, temperature=0.8)
print("Generated Text:", generated_text)